In [ ]:
import pandas as pd
import requests
import zipfile
import io
import os
import csv
import ctypes as ct
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib.dates import YearLocator
from matplotlib.ticker import PercentFormatter
import re
import seaborn as sns
import QuantLib as ql

from bgs.bgs_utils import clean_date, clean_percentage
from bgs.load_bgs_amounts import load_bgs_amounts
from bgs.load_gilt_details import load_csv_blocks
from bgs.load_bgs_prices import load_prices
from bgs.gilt_analytics import gilt_yield, yield_series, yield_curve, from_iso, yield_curves_pw, yield_curve_rv
from bgs.linker_analytics import linker_real_yield

%load_ext autoreload
%autoreload 2

### British Gilt Survey Amounts Table

In [ ]:
tables = load_bgs_amounts("downloads/BGSAmounts.csv")

conv = tables['Conventionals']
conv.replace("Redeemed", "", inplace=True)
conv = conv.apply(pd.to_numeric, errors='coerce')
conv.fillna(0, inplace=True)
conv.index = pd.Index([clean_date(x) for x in list(conv.index)]).to_period('M').to_timestamp('M')
conv.columns = [np.int64(x.strip()) for x in conv.columns]

old = tables['Calculated indexed nominal Old-style']
old.replace("Redeemed", "", inplace=True)
old = old.apply(pd.to_numeric, errors='coerce')
old.fillna(0, inplace=True)
old.index = pd.Index([clean_date(x) for x in list(old.index)]).to_period('M').to_timestamp('M')
old.columns = [np.int64(x.strip()) for x in old.columns]

new = tables['Calculated indexed nominal New-style']
new.replace("Redeemed", "", inplace=True)
new = new.apply(pd.to_numeric, errors='coerce')
new.fillna(0, inplace=True)
new.index = pd.Index([clean_date(x) for x in list(new.index)]).to_period('M').to_timestamp('M')
new.columns = [np.int64(x.strip()) for x in new.columns]

new_no_idx = tables['Index-linked New-style']
new_no_idx.replace("Redeemed", "", inplace=True)
new_no_idx = new_no_idx.apply(pd.to_numeric, errors='coerce')
new_no_idx.fillna(0, inplace=True)
new_no_idx.index = pd.Index([clean_date(x) for x in list(new_no_idx.index)]).to_period('M').to_timestamp('M')
new_no_idx.columns = [np.int64(x.strip()) for x in new_no_idx.columns]

### British Gilts Survey Month End Prices

In [ ]:
price_df=load_prices("downloads/BGSPrices.csv")
price_df.index = pd.to_datetime(price_df.index, format="%d %b %Y").to_period('M').to_timestamp('M')
for text in ['Amalgamated', 'Redeemed', 'redeemed']:
    price_df = price_df.replace(text, 0)
price_df = price_df.replace('missing', None)
price_df = price_df.ffill(axis=0)
price_df = price_df.fillna(0)
price_df.columns = [np.int64(x) for x in price_df.columns]

### British Gilts Survey Details Table (Bond Static Data, Coupon Maturity etc.) 

In [ ]:
details = load_csv_blocks("downloads/BGSDetails.csv")

conv_details = details['Conventionals']
conv_details['%'] = conv_details['%'].apply(clean_percentage)
conv_details['Sequence'] = conv_details['Sequence'].apply(lambda x: np.int64(x))

new_details = details['Index-Linked New-style']
new_details['%'] = new_details['%'].apply(clean_percentage)
new_details['Sequence'] = new_details['Sequence'].apply(lambda x: np.int64(x))

old_details = details['Index-Linked Old-style']
old_details['%'] = old_details['%'].apply(clean_percentage)
old_details['Sequence'] = old_details['Sequence'].apply(lambda x: np.int64(x))

strip_details = details['Strips']
strip_details['%'] = 0.0
strip_details['Sequence'] = strip_details['Sequence'].apply(lambda x: np.int64(x))

date_variables = ["Latest redemption date","Issue date","First coupon payable on date",]
for date_var in date_variables:
    conv_details[date_var] = conv_details[date_var].apply(clean_date)
    old_details[date_var] = old_details[date_var].apply(clean_date)
    new_details[date_var] = new_details[date_var].apply(clean_date)
strip_details['Latest redemption date'] = strip_details['Latest redemption date'].apply(clean_date)

In [ ]:
all_conv = conv_details[['Sequence', 'ISIN Code','%','Latest redemption date', 'Issue date', 'First coupon payable on date']]


### Fix a date

In [ ]:
def bgs_gilt_yield(bgs_id, calc_date):

    bond = conv_details[conv_details['Sequence']==bgs_id]
    issue_date=bond['Issue date'].dt.strftime("%Y-%m-%d").values[0]
    maturity_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
    first_cpn_date=bond['First coupon payable on date'].dt.strftime("%Y-%m-%d").values[0]
    last_cpn_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
    coupon=bond['%'].values[0]/100

    clean_price=price_df.loc[calc_date,bgs_id]


    return gilt_yield(
        trade_date=calc_date,
        issue_date=issue_date,
        maturity_date=maturity_date,
        first_cpn_date=first_cpn_date,
        last_cpn_date=last_cpn_date,
        clean_price=clean_price,
        coupon=coupon)
    

In [ ]:
calculation_date = "2025-04-30"


gilts_list = all_conv.loc[(all_conv['Latest redemption date'] > pd.to_datetime(calculation_date))&(all_conv['Issue date']< pd.to_datetime(calculation_date))]
gilts_list['yield'] = gilts_list['Sequence'].apply(lambda x: bgs_gilt_yield(bgs_id=x, calc_date=calculation_date))


f = plt.figure(figsize=(8, 5))
ax = f.add_subplot(1, 1, 1)

ax.xaxis.grid(True, "major", color="lightgray")
ax.yaxis.grid(True, "major", color="lightgray")
ax.xaxis.set_major_locator(YearLocator(10))
ax.set_ylim(0.03,0.055)
ax.yaxis.set_major_formatter(PercentFormatter(1.0))

sns.set_theme(style="darkgrid")

# use the scatterplot function to build the bubble map
sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="yield",
    ax=ax
)
plt.show()

In [ ]:
gilts_list['low_coupon'] = gilts_list.apply(lambda row: 1 if row['%'] < 2.0 else 0, axis=1)

In [ ]:
sns.set_theme(style="darkgrid")
f = plt.figure(figsize=(8, 5))
ax = f.add_subplot(1, 1, 1)

ax.xaxis.grid(True, "major", color="lightgray")
ax.yaxis.grid(True, "major", color="lightgray")
ax.xaxis.set_major_locator(YearLocator(10))
ax.set_ylim(0.03,0.055)
ax.yaxis.set_major_formatter(PercentFormatter(1.0))

sns.set_theme(style="darkgrid")
# use the scatterplot function to build the bubble map
ax = sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="yield",
    hue="low_coupon",
    ax=ax
)
ax.set_ylim(0.03,0.055)
plt.show()

In [ ]:
gilts_list['price'] = gilts_list['Sequence'].apply(lambda x: price_df[x].loc[calculation_date])

In [ ]:
c = yield_curve(calculation_date, gilts_list, method="Nelson/Siegel")

In [ ]:
c.maxDate()

In [ ]:
gilts_list['yield']

In [ ]:
f = plt.figure(figsize=(8, 5))
ax = f.add_subplot(1, 1, 1)

ax.xaxis.grid(True, "major", color="lightgray")
ax.yaxis.grid(True, "major", color="lightgray")
ax.xaxis.set_major_locator(YearLocator(10))

ax.yaxis.set_major_formatter(PercentFormatter(1.0))
styles = iter(["-", "--", ":", "-."])
ax.set_ylim(0.03,0.055)

dates = [from_iso(calculation_date) + ql.Period(i, ql.Months) for i in range(12 * 48 + 1)]

day_count = ql.ActualActual(ql.ActualActual.Bond)

rates = [
        c.zeroRate(d, day_count, ql.Continuous).rate() for d in dates
    ]
ax.plot_date(
    [d.to_date() for d in dates],
    rates,
    next(styles)
)
ax.legend(loc="best")

sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="yield",
    hue="low_coupon",
    ax=ax
)

In [ ]:
high_coupon = gilts_list[gilts_list['low_coupon']==0]
high_coupon_curve = yield_curve(calculation_date, high_coupon, method="Nelson/Siegel")

In [ ]:
f = plt.figure(figsize=(8, 5))
ax = f.add_subplot(1, 1, 1)

ax.xaxis.grid(True, "major", color="lightgray")
ax.yaxis.grid(True, "major", color="lightgray")
ax.xaxis.set_major_locator(YearLocator(10))

ax.yaxis.set_major_formatter(PercentFormatter(1.0))
styles = iter(["-", "--", ":", "-."])
ax.set_ylim(0.03,0.055)

dates = [from_iso(calculation_date) + ql.Period(i, ql.Months) for i in range(12 * 43 + 1)]

day_count = ql.ActualActual(ql.ActualActual.Bond)


rates = [
    high_coupon_curve.zeroRate(d, day_count, ql.Continuous).rate() for d in dates
]
ax.plot_date(
    [d.to_date() for d in dates],
    rates,
    next(styles),
)
ax.legend(loc="best")

sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="yield",
    hue="low_coupon",
    ax=ax
)

In [ ]:
low_coupon = gilts_list[gilts_list['low_coupon']==1]
low_coupon_curve = yield_curve(calculation_date, low_coupon)

In [ ]:
f = plt.figure(figsize=(8, 5))
ax = f.add_subplot(1, 1, 1)

ax.xaxis.grid(True, "major", color="lightgray")
ax.yaxis.grid(True, "major", color="lightgray")
ax.xaxis.set_major_locator(YearLocator(10))
ax.set_ylim(0.03,0.055)

ax.yaxis.set_major_formatter(PercentFormatter(1.0))
styles = iter(["-", "--", ":", "-."])

dates = [from_iso(calculation_date) + ql.Period(i, ql.Months) for i in range(12 * 43 + 1)]

day_count = ql.ActualActual(ql.ActualActual.Bond)

rates = [
    low_coupon_curve.zeroRate(d, day_count, ql.Continuous).rate() for d in dates
]
ax.plot_date(
    [d.to_date() for d in dates],
    rates,
    next(styles),
)
ax.legend(loc="best")

sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="yield",
    hue="low_coupon",
    ax=ax
)

In [ ]:
gilts_list.loc[gilts_list['yield']==gilts_list['yield'].min()]

In [ ]:
rv_df = yield_curve_rv(calculation_date, gilts_list)

In [ ]:
rv_df.columns

In [ ]:
(rv_df['yield'] - rv_df['curve_yield']).mean()

In [ ]:
(rv_df['price'] - rv_df['curve_price']).abs().mean()

In [ ]:
(rv_df['yield'] - rv_df['curve_yield']).abs().mean()

In [ ]:
rv_df.columns

In [ ]:
c = yield_curve(calculation_date, gilts_list, method="Nelson/Siegel")

In [ ]:
f = plt.figure(figsize=(8, 5))
ax = f.add_subplot(1, 1, 1)

ax.xaxis.grid(True, "major", color="lightgray")
ax.yaxis.grid(True, "major", color="lightgray")
ax.xaxis.set_major_locator(YearLocator(10))
ax.set_ylim(0.03,0.055)

ax.yaxis.set_major_formatter(PercentFormatter(1.0))
styles = iter(["-", "--", ":", "-."])

sns.scatterplot(
    data=rv_df,
    x="Latest redemption date",
    y="curve_yield",
    ax=ax
)

dates = [from_iso(calculation_date) + ql.Period(i, ql.Months) for i in range(12 * 48 + 1)]

day_count = ql.ActualActual(ql.ActualActual.Bond)

rates = [
        c.zeroRate(d, day_count, ql.Continuous).rate() for d in dates
    ]
ax.plot_date(
    [d.to_date() for d in dates],
    rates,
    next(styles)
)
ax.legend(loc="best")